In [1]:
from z3 import *

In [2]:
# create uninterpreted sort Node and Epoch
Node = DeclareSort('Node')
Epoch = DeclareSort('Epoch')

In [3]:
class DistLockState():
    def __init__(self, name):
        self.name = name

        # relations
        self.ep = Function(f'{name}.ep', Node, Epoch)
        self.held = Function(f'{name}.held', Node, BoolSort())
        self.transfer = Function(f'{name}.transfer', Epoch, Node, BoolSort())
        self.locked = Function(f'{name}.locked', Epoch, Node, BoolSort())

DistLockState('test_pre') # only for testing

In [4]:
class DistLockModel():
    def __init__(self):
        # constants
        self.le = Function(f'le', Epoch, Epoch, BoolSort())
        self.zero = Const(f'zero', Epoch)
        self.one = Const(f'one', Epoch)
        self.first = Const(f'first', Node)

        self.states = {}
    
    def get_state(self, name):
        if name not in self.states:
            self.states[name] = DistLockState(name)
        return self.states[name]
    
    def get_axioms(self):
        e1, e2, e3 = Consts('e1 e2 e3', Epoch)
        
        Axioms = ForAll([e1, e2, e3],
            And(
                # reflexivity
                self.le(e1, e1),
                # transitivity
                Implies(And(self.le(e1, e2), self.le(e2, e3)), self.le(e1, e3)),
                # antisymmetry
                Implies(And(self.le(e1, e2), self.le(e2, e1)), e1 == e2),
                # totality
                Or(self.le(e1, e2), self.le(e2, e1)),

                # zero
                self.le(self.zero, e1),
                self.one != self.zero,
            ),
        )

        return Axioms
    
    def get_interp(self, model: ModelRef):
        # create a dict of all the functions
        interp = {}
        for f in model.decls():
            interp[f.name()] = model.get_interp(f)
        
        return interp

M = DistLockModel()
M.get_axioms() # only for testing

ForAll([e1, e2, e3],
       And(le(e1, e1),
           Implies(And(le(e1, e2), le(e2, e3)), le(e1, e3)),
           Implies(And(le(e1, e2), le(e2, e1)), e1 == e2),
           Or(le(e1, e2), le(e2, e1)),
           le(zero, e1),
           one != zero))

In [5]:
# ep1 = Function('ep1', Node, Epoch)
# ep2 = Function('ep2', Node, Epoch)

# held1 = Function('held1', Node, BoolSort())
# held2 = Function('held2', Node, BoolSort())

# transfer1 = Function('transfer1', Epoch, Node, BoolSort())
# transfer2 = Function('transfer2', Epoch, Node, BoolSort())

# locked1 = Function('locked1', Epoch, Node, BoolSort())
# locked2 = Function('locked2', Epoch, Node, BoolSort())

In [6]:
# Inv: le(E1, E2) & E1 ~= E2 -> le(E1,ep(N1)) | ~le(E2,ep(N1))

def get_inv1(M: DistLockModel, S: DistLockState):
    e1, e2 = Consts('e1 e2', Epoch)
    n1 = Const('n1', Node)

    Inv = ForAll([e1, e2, n1],
        Implies(
            And(
                M.le(e1, e2),
                e1 != e2
            ),
            Or(
                M.le(e1, S.ep(n1)),
                Not(
                    M.le(e2, S.ep(n1))
                )
            )
        )
    )
    return Inv

M = DistLockModel()
S = M.get_state('pre')
get_inv1(M, S) # only for testing

ForAll([e1, e2, n1],
       Implies(And(le(e1, e2), e1 != e2),
               Or(le(e1, pre.ep(n1)),
                  Not(le(e2, pre.ep(n1))))))

In [7]:
# Inv: safety property locked(E, N1) & locked(E, N2) -> N1 = N2

def get_safety_inv(M: DistLockModel, S: DistLockState):
    e1 = Const('e1', Epoch)
    n1, n2 = Consts('n1 n2', Node)

    Inv = ForAll([e1, n1, n2],
        Implies(
            And(
                S.locked(e1, n1),
                S.locked(e1, n2)
            ),
            n1 == n2
        )
    )

    return Inv

M = DistLockModel()
S = M.get_state('pre')
get_safety_inv(M, S) # only for testing

ForAll([e1, n1, n2],
       Implies(And(pre.locked(e1, n1), pre.locked(e1, n2)),
               n1 == n2))

In [8]:
# Accept action

def get_acceptance_action(M: DistLockModel, S1: DistLockState, S2: DistLockState):
    e1 = Const('e1', Epoch)
    n1 = Const('n1', Node)

    AcceptAction = ForAll([n1, e1],
        Implies(
            And(
                S1.transfer(e1, n1),
                Not(M.le(e1, S1.ep(n1)))
            ),
            And(
                S2.held(n1),
                S2.ep(n1) == e1,
                S2.locked(e1, n1)
            )
        )
    )

    return AcceptAction

M = DistLockModel()
S1 = M.get_state('pre')
S2 = M.get_state('post')
get_acceptance_action(M, S1, S2) # only for testing

ForAll([n1, e1],
       Implies(And(pre.transfer(e1, n1),
                   Not(le(e1, pre.ep(n1)))),
               And(post.held(n1),
                   post.ep(n1) == e1,
                   post.locked(e1, n1))))

In [9]:
def get_inductiveness_vc(M: DistLockModel, S1: DistLockState, action_fn, S2: DistLockState, inv_fn):
    vc = Not(
        Implies(
            M.get_axioms(),
            Implies(
                And(
                    inv_fn(M, S1),
                    action_fn(M, S1, S2),
                ),
                inv_fn(M, S2)
            )
        )
    )
    return vc

M = DistLockModel()
S1 = M.get_state('pre')
S2 = M.get_state('post')
vc = get_inductiveness_vc(M, S1, get_acceptance_action, S2, get_inv1)
print(vc.sexpr()) # only for testing

(let ((a!1 (forall ((e1 Epoch) (e2 Epoch) (e3 Epoch))
             (and (le e1 e1)
                  (=> (and (le e1 e2) (le e2 e3)) (le e1 e3))
                  (=> (and (le e1 e2) (le e2 e1)) (= e1 e2))
                  (or (le e1 e2) (le e2 e1))
                  (le zero e1)
                  (distinct one zero))))
      (a!2 (forall ((e1 Epoch) (e2 Epoch) (n1 Node))
             (let ((a!1 (or (le e1 (pre.ep n1)) (not (le e2 (pre.ep n1))))))
               (=> (and (le e1 e2) (distinct e1 e2)) a!1))))
      (a!3 (forall ((n1 Node) (e1 Epoch))
             (let ((a!1 (and (pre.transfer e1 n1) (not (le e1 (pre.ep n1))))))
               (=> a!1
                   (and (post.held n1) (= (post.ep n1) e1) (post.locked e1 n1))))))
      (a!4 (forall ((e1 Epoch) (e2 Epoch) (n1 Node))
             (let ((a!1 (or (le e1 (post.ep n1)) (not (le e2 (post.ep n1))))))
               (=> (and (le e1 e2) (distinct e1 e2)) a!1)))))
  (not (=> a!1 (=> (and a!2 a!3) a!4))))


In [10]:
M = DistLockModel()
S1 = M.get_state('pre')
S2 = M.get_state('post')

solver = Solver()
solver.add(get_inductiveness_vc(M, S1, get_acceptance_action, S2, get_inv1))
solver.check()

unsat

In [11]:
M = DistLockModel()
S1 = M.get_state('pre')
S2 = M.get_state('post')

solver = Solver()
solver.add(get_inductiveness_vc(M, S1, get_acceptance_action, S2, get_safety_inv))
solver.check()

sat

In [12]:
m = solver.model()
print(m.sexpr())

;; universe for Epoch:
;;   Epoch!val!2 Epoch!val!1 Epoch!val!0 
;; -----------
;; definitions for universe elements:
(declare-fun Epoch!val!2 () Epoch)
(declare-fun Epoch!val!1 () Epoch)
(declare-fun Epoch!val!0 () Epoch)
;; cardinality constraint:
(forall ((x Epoch)) (or (= x Epoch!val!2) (= x Epoch!val!1) (= x Epoch!val!0)))
;; -----------
;; universe for Node:
;;   Node!val!0 Node!val!1 
;; -----------
;; definitions for universe elements:
(declare-fun Node!val!0 () Node)
(declare-fun Node!val!1 () Node)
;; cardinality constraint:
(forall ((x Node)) (or (= x Node!val!0) (= x Node!val!1)))
;; -----------
(define-fun zero () Epoch
  Epoch!val!2)
(define-fun one () Epoch
  Epoch!val!1)
(define-fun post.held ((x!0 Node)) Bool
  (= x!0 Node!val!1))
(define-fun pre.ep ((x!0 Node)) Epoch
  Epoch!val!1)
(define-fun post.ep ((x!0 Node)) Epoch
  (ite (= x!0 Node!val!1) Epoch!val!0
    Epoch!val!1))
(define-fun post.locked ((x!0 Epoch) (x!1 Node)) Bool
  (or (and (not (= x!0 Epoch!val!1))
   

In [13]:
M.get_interp(m)

{'zero': Epoch!val!2,
 'one': Epoch!val!1,
 'post.held': [else -> Var(0) == Node!val!1],
 'pre.ep': [else -> Epoch!val!1],
 'post.ep': [Node!val!1 -> Epoch!val!0, else -> Epoch!val!1],
 'post.locked': [else ->
  Or(And(Not(Var(0) == Epoch!val!1),
         Not(Var(0) == Epoch!val!2),
         Var(1) == Node!val!1),
     And(Not(Var(0) == Epoch!val!1),
         Not(Var(0) == Epoch!val!2),
         Not(Var(1) == Node!val!1)))],
 'pre.transfer': [else -> False],
 'pre.locked': [else -> False],
 'le': [else ->
  Or(And(Not(Var(0) == Epoch!val!1),
         Not(Var(0) == Epoch!val!2),
         Var(1) == Epoch!val!1,
         Not(Var(1) == Epoch!val!2)),
     And(Var(0) == Epoch!val!2,
         Var(1) == Epoch!val!1,
         Not(Var(1) == Epoch!val!2)),
     And(Not(Var(0) == Epoch!val!1),
         Not(Var(0) == Epoch!val!2),
         Not(Var(1) == Epoch!val!1),
         Not(Var(1) == Epoch!val!2)),
     And(Var(0) == Epoch!val!1,
         Not(Var(0) == Epoch!val!2),
         Var(1) == Epoch!